In [1]:
%run NER_common.ipynb

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import seqeval.metrics
from tqdm import tqdm_notebook

import numpy as np
from itertools import chain, islice
from collections import Counter
from collections import defaultdict
from functools import partial

In [3]:
from tagging import *

In [4]:
import pytorch_transformers
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader
import pytorch_lightning as pl
from test_tube import Experiment
import argparse
import os.path

In [5]:
def bert_preprocessing(inputs, targets, bert_tokenizer, bert_specials=None):
    new_inputs = []
    new_targets = []
    masks = []
#     bert_tokenizer = pytorch_transformers.BertTokenizer.from_pretrained('bert-base-uncased')
    for text, tags in tqdm_notebook(zip(inputs, targets), total=len(inputs)):
        new_tokens, new_tags = [],[]
        mask = []
        for token, tag in zip(text, tags):
            if bert_specials and token in bert_specials:
                token_pieces = [bert_specials[token]]
            else:
                token_pieces = bert_tokenizer.tokenize(token)
            new_tokens.extend(token_pieces)
            new_tags.extend(split_tag(tag, len(token_pieces)))
            if experiment_config.mask_additional_wordpieces:
                mask.extend([True] + [False] * (len(token_pieces) - 1))
            else:
                mask.extend([True] * len(token_pieces))
        new_inputs.append(new_tokens)
        new_targets.append(new_tags)
        masks.append(mask)
    return new_inputs, new_targets, masks

def bert_numericalize_targets(targets, target_vocab):
    target_ids = target_vocab.numericalize(targets)
    return [[0] + ids + [0] for ids in target_ids]

def bert_numericalize_mask(masks):
    return [[False] + mask + [False] for mask in masks]

def bert_numericalize_inputs(inputs, bert_tokenizer):
    result = []
    for input_tokens in inputs:
        ids = [bert_tokenizer._convert_token_to_id(tok) for tok in input_tokens]
        ids = bert_tokenizer.add_special_tokens_single_sentence(ids)
        result.append(ids)
    return result

def bert_numericalize(inputs, targets, masks, bert_tokenizer, target_vocab):
    input_ids = bert_numericalize_inputs(inputs, bert_tokenizer)
    target_ids = bert_numericalize_targets(targets, target_vocab)
    mask_ids = bert_numericalize_mask(masks)
    return input_ids, target_ids, mask_ids

In [6]:
experiment_config = argparse.Namespace()
experiment_config.basic_model = 'BiLSTM-BERT'

experiment_config.replace_urls = False
experiment_config.replace_numbers = False 
experiment_config.split_hashtags = False # оставить False, не реализовано, BertTokenizer сам это сделает
experiment_config.split_nicknames = False # оставить False, не реализовано,  BertTokenizer сам это сделает

experiment_config.mask_additional_wordpieces = True # оставить True, не реализовано
experiment_config.bert_n_last_layers = 4
experiment_config.bert_pretrained_name = 'bert-base-cased'
experiment_config.val_batch_size = 24
experiment_config.train_batch_size = 16
experiment_config.lr = 2e-4
experiment_config.gradient_acccumulation_steps = 1
experiment_config.gradient_clipping_norm = 5.0
experiment_config.n_epochs = 12
experiment_config.emb_dropout_type = 'locked'
experiment_config.emb_dropout = 0.5
experiment_config.bilstm_hidden_size = 1024
experiment_config.bilstm_dropout = 0.2
experiment_config.bilstm_n_layers = 1
experiment_config.pre_fc_dropout = 0.2
# experiment_config.mask_additional_wordpieces = True

experiment_dir = 'NER_experiments/'
experiment_config.experiment_name = 'BILSTM-over-BERT'

In [7]:
original_inputs, original_targets = read_data('data/data.txt')
unique_tags = sorted(count_tags(original_targets).keys(), key=lambda tag: tag[2:])
experiment_config.n_classes = len(unique_tags)

In [8]:
bert_tokenizer = pytorch_transformers.BertTokenizer.from_pretrained(experiment_config.bert_pretrained_name, do_lower_case=False)

In [9]:
bert_specials = {'<URL>': '[unused1]', '<NUM>': '[unused2]'}

In [10]:
target_vocab = Vocab.from_id2word(unique_tags, unk_index=None, n_specials=0)

In [11]:
inputs, targets, _ = basic_preprocessing(original_inputs, original_targets,
                                      replace_urls=experiment_config.replace_urls,
                                      replace_numbers=experiment_config.replace_numbers,
                                      split_hashtags=False,
                                      split_mentions=False)

In [12]:
bert_tokenized_inputs, bert_tokenized_targets, bert_masks = bert_preprocessing(inputs, targets, bert_tokenizer, bert_specials)

In [13]:
split_names = ['train', 'val', 'test']

In [14]:
original_inputs_split, original_targets_split = split_to_dicts(
    [original_inputs, original_targets],
    (train_indices, val_indices, test_indices), 
    split_names)

bert_inputs_split, bert_targets_split, bert_masks_split = split_to_dicts(
    [bert_tokenized_inputs, bert_tokenized_targets, bert_masks],
    (train_indices, val_indices, test_indices), 
    split_names)

In [15]:
bert_numericalized_inputs, bert_numericalized_targets, bert_numericalized_masks = bert_numericalize(
    bert_tokenized_inputs, 
    bert_tokenized_targets,
    bert_masks, bert_tokenizer, target_vocab)

In [16]:
bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl = split_to_dicts([bert_numericalized_inputs, bert_numericalized_targets, bert_numericalized_masks],
                                                                            (train_indices, val_indices, test_indices), split_names)

In [17]:
class BertEncoderDataset(Dataset):
    def __init__(self, input_ids, target_ids, mask_ids):
        self.inputs = input_ids
        self.targets = target_ids
        self.masks = mask_ids
        assert len(input_ids) == len(target_ids) == len(self.masks)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return (self.inputs[idx], self.targets[idx], self.masks[idx])
    
    @staticmethod
    def collate(examples):
        inputs, targets, masks, lengths = [],[],[],[]
        for inp, tgt, msk in examples:
            inputs.append(torch.tensor(inp, dtype=torch.long))
            targets.append(torch.tensor(tgt, dtype=torch.long))
            masks.append(torch.tensor(msk, dtype=torch.bool))
            lengths.append(len(inp))
            
        input_tensor = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True)
        target_tensor = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True)
        mask_tensor = torch.nn.utils.rnn.pad_sequence(masks, batch_first=True)
        lengths_tensor = torch.tensor(lengths)
#         print(input_tensor.size(), target_tensor.size(), mask_tensor.size())
        return input_tensor, target_tensor, mask_tensor, lengths_tensor

In [18]:
train_dataset = BertEncoderDataset(*[spl['train'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])
val_dataset = BertEncoderDataset(*[spl['val'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])
test_dataset = BertEncoderDataset(*[spl['test'] for spl in (bert_input_ids_spl, bert_target_ids_spl, bert_mask_ids_spl)])

### Model

In [19]:
class BertEncoder(Encoder):
    def __init__(self, config):
        super().__init__()
        self.bert = pytorch_transformers.BertModel.from_pretrained(
            config.bert_pretrained_name,
            output_hidden_states=True)
        self.bert.eval()
        self.n_last_layers = config.bert_n_last_layers
        config.embedding_dim = self.bert.config.hidden_size * self.n_last_layers
        
    def forward(self, encoder_inputs):
        self.bert.eval()
        input_tensor = encoder_inputs[0]
        pad_mask = (input_tensor != 0).type(torch.float32)
        with torch.no_grad():
            bert_layer_outputs = self.bert(input_tensor, attention_mask=pad_mask)[2][-self.n_last_layers:]
        concatenated = torch.cat(bert_layer_outputs, dim=-1)
        return concatenated

In [20]:
class BiLSTM_BERT_Lightning(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        encoder = BertEncoder(config)
        tagger = BiLSTMTagger(config)
        self.model = TaggerWithEncoder(encoder, tagger)
        
    def encoder_tagger_inputs(self, input_tensor, length_tensor):
        return (input_tensor,), (length_tensor,)
        
    def forward(self, input_tensor, length_tensor):
        encoder_inputs, tagger_inputs = self.encoder_tagger_inputs(input_tensor, length_tensor)
        
        return self.model(encoder_inputs, tagger_inputs)
    
    def compute_loss_on_batch(self, batch):
        input_tensor, target_tensor, mask_tensor, lengths_tensor = batch
        logits = self(input_tensor, lengths_tensor)
        
        loss_mask = ~mask_tensor
        loss = masked_crossentropy_loss(logits, target_tensor, loss_mask)
        return loss
    
    def training_step(self, batch, batch_nb):
        # REQUIRED
        self.train()
        loss = self.compute_loss_on_batch(batch)
        return {'loss': loss}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        loss = self.compute_loss_on_batch(batch)
        input_tensor, target_tensor, mask_tensor, lengths_tensor =  batch
        encoder_inputs, tagger_inputs = self.encoder_tagger_inputs(input_tensor, lengths_tensor)
         # encoder_inputs, tagger_inputs, mask_tensor, target_vocab
        predicted_tags = self.model.predict_tags(encoder_inputs, tagger_inputs, mask_tensor, target_vocab)
        
        return {'val_loss': loss, 'tags': predicted_tags}

    def validation_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.tensor([x['val_loss'] for x in outputs]).mean()
        predictions = list(chain.from_iterable(x['tags'] for x in outputs))
        f1_score = seqeval.metrics.f1_score(original_targets_split['val'], predictions)
        
        print(seqeval.metrics.classification_report(original_targets_split['val'], predictions))
        metrics = {'avg_val_loss': avg_loss.item(), 'f1': f1_score}
        metrics_to_write = dict(metrics, epoch=self.trainer.current_epoch+1)
#         metrics.update(self.trainer.tng_tqdm_dic)
        
#         scalar_metrics = self.trainer.__metrics_to_scalars(
#                     metrics, blacklist=self.trainer.__log_vals_blacklist())
        
        assert self.experiment
        self.experiment.log(metrics_to_write)
        self.experiment.save()
        
        return metrics
    
    def predict_loader(self, loader, device='cuda'):
        result = []
        self.eval()
        with torch.no_grad():
            for batch in loader:
                input_tensor, target_tensor, mask_tensor, lengths_tensor =  batch
                lengths_tensor = lengths_tensor.to(device)
                mask_tensor = mask_tensor.to(device)
                input_tensor = input_tensor.to(device)
                encoder_inputs, tagger_inputs = self.encoder_tagger_inputs(input_tensor, lengths_tensor)
                
                predicted_tags = self.model.predict_tags(encoder_inputs, tagger_inputs, mask_tensor, target_vocab)
                result.extend(predicted_tags)
        
        return result
    
    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        return torch.optim.Adam(self.parameters(), lr=self.config.lr)

    @pl.data_loader
    def tng_dataloader(self):
        # REQUIRED
        assert isinstance(train_dataset, BertEncoderDataset)
        return DataLoader(train_dataset, batch_size=self.config.train_batch_size, shuffle=True, collate_fn=BertEncoderDataset.collate)

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        assert isinstance(val_dataset, BertEncoderDataset)
        return DataLoader(val_dataset, batch_size=self.config.val_batch_size, shuffle=False, collate_fn=BertEncoderDataset.collate)

In [21]:
def model_test():
    bilstm = BiLSTM_BERT_Lightning(experiment_config)
    bilstm = bilstm.cuda()
    predictions = bilstm.predict_loader(bilstm.val_dataloader)
    print(len(predictions))
    print(predictions[100])
    del bilstm
    torch.cuda.empty_cache()

In [22]:
exp = Experiment(save_dir=experiment_dir, name=experiment_config.experiment_name)

In [23]:
bilstm_model = BiLSTM_BERT_Lightning(experiment_config)

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [24]:
exp.argparse(experiment_config)
exp.save()

In [25]:
bilstm_model = bilstm_model.cuda()

In [26]:
checkpoint_path = f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/checkpoint'

In [27]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    save_best_only=True,
    verbose=True,
    monitor='f1',
    mode='max'
)

early_stop = pl.callbacks.EarlyStopping(
        monitor='f1',
        patience=5,
        verbose=True,
        mode='max'
)

In [28]:
trainer = pl.Trainer(experiment=exp,
                     max_nb_epochs=experiment_config.n_epochs,
                     gpus=[0],
                     gradient_clip=experiment_config.gradient_clipping_norm,
                     early_stop_callback=early_stop,
                     accumulate_grad_batches=experiment_config.gradient_acccumulation_steps,
                     add_log_row_interval=100,
                     checkpoint_callback=checkpoint_callback)

VISIBLE GPUS: '0'
gpu available: True, used: True


In [29]:
trainer.fit(bilstm_model)

  0%|          | 0/5 [00:00<?, ?it/s]

                                              Name               Type  \
0                                            model  TaggerWithEncoder   
1                                    model.encoder        BertEncoder   
2                               model.encoder.bert          BertModel   
3                    model.encoder.bert.embeddings     BertEmbeddings   
4    model.encoder.bert.embeddings.word_embeddings          Embedding   
..                                             ...                ...   
218                                   model.tagger       BiLSTMTagger   
219                       model.tagger.emb_dropout      LockedDropout   
220                               model.tagger.rnn               LSTM   
221                    model.tagger.pre_fc_dropout            Dropout   
222                                model.tagger.fc             Linear   

        Params  
0    123020053  
1    108310272  
2    108310272  
3     22665216  
4     22268928  
..         ...  
218 

  0%|          | 1/361 [00:00<01:02,  5.76it/s, batch_nb=0, epoch=0, gpu=0, loss=3.167, v_nb=3]

             precision    recall  f1-score   support

        loc       0.00      0.00      0.00       156
     person       0.01      0.02      0.01       131
      other       0.00      0.00      0.00       110
    product       0.00      0.00      0.00        38
musicartist       0.00      0.00      0.00        33
    company       0.00      0.00      0.00        84
 sportsteam       0.00      0.00      0.00        21
   facility       0.00      0.00      0.00        58
     tvshow       0.00      0.00      0.00        11
      movie       0.00      0.00      0.00        17

  micro avg       0.00      0.00      0.00       659
  macro avg       0.00      0.00      0.00       659



100%|██████████| 361/361 [00:27<00:00,  7.37it/s, avg_val_loss=0.185, batch_nb=324, epoch=0, f1=0.437, gpu=0, loss=0.208, v_nb=3]

...

  0%|          | 1/361 [00:00<00:50,  7.14it/s, avg_val_loss=0.155, batch_nb=0, epoch=11, f1=0.596, gpu=0, loss=0.027, v_nb=3]    

             precision    recall  f1-score   support

        loc       0.68      0.74      0.71       156
     person       0.72      0.78      0.75       131
      other       0.40      0.45      0.43       110
    product       0.30      0.37      0.33        38
musicartist       0.60      0.45      0.52        33
    company       0.69      0.62      0.65        84
 sportsteam       0.56      0.67      0.61        21
   facility       0.56      0.53      0.55        58
     tvshow       0.20      0.18      0.19        11
      movie       0.36      0.24      0.29        17

  micro avg       0.58      0.61      0.60       659
  macro avg       0.59      0.61      0.60       659

save callback...

Epoch 00011: f1 did not improve


100%|██████████| 361/361 [00:28<00:00,  7.18it/s, avg_val_loss=0.158, batch_nb=324, epoch=11, f1=0.615, gpu=0, loss=0.025, v_nb=3]

             precision    recall  f1-score   support

        loc       0.68      0.76      0.72       156
     person       0.73      0.79      0.75       131
      other       0.41      0.46      0.44       110
    product       0.42      0.42      0.42        38
musicartist       0.62      0.39      0.48        33
    company       0.68      0.62      0.65        84
 sportsteam       0.54      0.67      0.60        21
   facility       0.60      0.64      0.62        58
     tvshow       0.25      0.18      0.21        11
      movie       0.62      0.29      0.40        17

  micro avg       0.61      0.63      0.62       659
  macro avg       0.61      0.63      0.61       659

save callback...

Epoch 00012: f1 did not improve


1

### Inference

In [30]:
import os

In [31]:
checkpoint_file = os.listdir(checkpoint_path)[0]
assert checkpoint_file.startswith('_ckpt_')
checkpoint_file_path = checkpoint_path + '/' + checkpoint_file
print(checkpoint_file_path)

tags_path = f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/meta_tags.csv'
print(tags_path)

NER_experiments//BILSTM-over-BERT/version_3/checkpoint/_ckpt_epoch_9.ckpt
NER_experiments//BILSTM-over-BERT/version_3/meta_tags.csv


In [32]:
device = 'cuda'

In [33]:
bilstm_model = BiLSTM_BERT_Lightning.load_from_metrics(checkpoint_file_path, tags_path, on_gpu=False)
bilstm_model.freeze()
bilstm_model = bilstm_model.to(device)

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [34]:
test_loader = DataLoader(test_dataset, batch_size=bilstm_model.config.val_batch_size, collate_fn=test_dataset.collate)

In [35]:
test_pred = bilstm_model.predict_loader(test_loader, device)

In [36]:
print(test_pred[0], original_targets_split['test'][0], sep='\n')

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [37]:
report = seqeval.metrics.classification_report(original_targets_split['test'], test_pred, digits=3)

In [38]:
test_report_save_path =f'{experiment_dir}/{experiment_config.experiment_name}/version_{exp.version}/test_report.txt'

In [39]:
with open(test_report_save_path, 'w+') as of:
    of.write(report)

In [40]:
def predict_for_tokens(model: BiLSTM_BERT_Lightning, tokens, device='cuda'):
    fake_targets = [['O' for _ in text] for text in tokens]
    preproc_tokens, preproc_targets, _ = basic_preprocessing(tokens, fake_targets, 
                                                             replace_urls=experiment_config.replace_urls,
                                                              replace_numbers=experiment_config.replace_numbers,
                                                              split_hashtags=experiment_config.split_hashtags,
                                                              split_mentions=experiment_config.split_nicknames)
    preproc_tokens, preproc_targets, preproc_mask = bert_preprocessing(preproc_tokens, preproc_targets, bert_tokenizer, bert_specials)
    input_ids, target_ids, mask_ids = bert_numericalize(preproc_tokens, preproc_targets, preproc_mask, bert_tokenizer, target_vocab)
    
    batch = BertEncoderDataset.collate(zip(input_ids, target_ids, mask_ids))
    input_tensor, target_tensor, mask_tensor, lengths_tensor = batch
    
    input_tensor, mask_tensor, lengths_tensor = input_tensor.to(device), mask_tensor.to(device), lengths_tensor.to(device)
    encoder_inputs, tagger_inputs = model.encoder_tagger_inputs(input_tensor, lengths_tensor)
    
    model.eval()
    with torch.no_grad():
        tags = bilstm_model.model.predict_tags(encoder_inputs, tagger_inputs, mask_tensor, target_vocab)
        
    return tags

In [41]:
text_example = "Satellite imagery this morning of now Category 5 Hurricane Dorian approaching the Abaco Islands in the northern Bahamas. For the latest on Dorian visit http://hurricanes.gov"

In [42]:
import nltk

In [43]:
print(text_example)

Satellite imagery this morning of now Category 5 Hurricane Dorian approaching the Abaco Islands in the northern Bahamas. For the latest on Dorian visit http://hurricanes.gov


In [44]:
tokens_example = nltk.tokenize.TweetTokenizer().tokenize(text_example)
print(tokens_example)

['Satellite', 'imagery', 'this', 'morning', 'of', 'now', 'Category', '5', 'Hurricane', 'Dorian', 'approaching', 'the', 'Abaco', 'Islands', 'in', 'the', 'northern', 'Bahamas', '.', 'For', 'the', 'latest', 'on', 'Dorian', 'visit', 'http://hurricanes.gov']


In [45]:
prediction = predict_for_tokens(bilstm_model, [tokens_example])[0]

In [46]:
print( len(prediction), len(tokens_example))

26 26


In [47]:
for token, tag in zip(tokens_example, prediction):
    print(token, tag)

Satellite O
imagery O
this O
morning O
of O
now O
Category O
5 O
Hurricane O
Dorian O
approaching O
the O
Abaco B-geo-loc
Islands I-geo-loc
in O
the O
northern B-geo-loc
Bahamas B-geo-loc
. O
For O
the O
latest O
on O
Dorian B-person
visit O
http://hurricanes.gov O
